In [0]:
spark

In [0]:
from pyspark.sql import SparkSession
 #create spark session
spark = SparkSession.builder.appName("Stock Analysis").getOrCreate()

In [0]:
spark

In [0]:
stock=spark.read.csv("/FileStore/tables/ALL_STOCKS.csv",header=True,inferSchema=True)

In [0]:
# show the scema of data 
stock.printSchema()

root
 |-- Ticker: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Close/Last: string (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)



In [0]:
stock.show(5)

+------+----------+----------+--------+--------+--------+--------+
|Ticker|      Date|Close/Last|  Volume|    Open|    High|     Low|
+------+----------+----------+--------+--------+--------+--------+
|  AMZN|2023-05-31|  $120.58 |72800790|$121.45 |$122.04 |$119.17 |
|  AMZN|2023-05-30|  $121.66 |64314810|$122.37 |$122.92 |$119.86 |
|  AMZN|2023-05-26|  $120.11 |96779890|$116.04 |$121.50 |$116.02 |
|  AMZN|2023-05-25|  $115.00 |66496680|$116.63 |$116.87 |$114.31 |
|  AMZN|2023-05-24|  $116.75 |63487940|$115.35 |$117.34 |$115.02 |
+------+----------+----------+--------+--------+--------+--------+
only showing top 5 rows


In [0]:
stock.select("Ticker").show(5)

+------+
|Ticker|
+------+
|  AMZN|
|  AMZN|
|  AMZN|
|  AMZN|
|  AMZN|
+------+
only showing top 5 rows


In [0]:

stock.filter(stock.Ticker == "MSFT").show(5)

+------+----------+----------+--------+--------+--------+--------+
|Ticker|      Date|Close/Last|  Volume|    Open|    High|     Low|
+------+----------+----------+--------+--------+--------+--------+
|  MSFT|2023-05-31|  $328.39 |45950550|$332.29 |$335.94 |$327.33 |
|  MSFT|2023-05-30|  $331.21 |29503070|$335.23 |$335.74 |$330.52 |
|  MSFT|2023-05-26|  $332.89 |36630630|$324.02 |$333.40 |$323.88 |
|  MSFT|2023-05-25|  $325.92 |43301740|$323.24 |$326.90 |$320.00 |
|  MSFT|2023-05-24|  $313.85 |23384890|$314.73 |$316.50 |$312.61 |
+------+----------+----------+--------+--------+--------+--------+
only showing top 5 rows


In [0]:
stock.filter((stock.Ticker =="AMZN")&(stock.Date=="2023-05-31")).show()

+------+----------+----------+--------+--------+--------+--------+
|Ticker|      Date|Close/Last|  Volume|    Open|    High|     Low|
+------+----------+----------+--------+--------+--------+--------+
|  AMZN|2023-05-31|  $120.58 |72800790|$121.45 |$122.04 |$119.17 |
+------+----------+----------+--------+--------+--------+--------+



In [0]:
stock.filter((stock.Ticker.isin(["AMZN","MSFT"]))&(stock.Date=="2023-05-31")).show()

+------+----------+----------+--------+--------+--------+--------+
|Ticker|      Date|Close/Last|  Volume|    Open|    High|     Low|
+------+----------+----------+--------+--------+--------+--------+
|  AMZN|2023-05-31|  $120.58 |72800790|$121.45 |$122.04 |$119.17 |
|  MSFT|2023-05-31|  $328.39 |45950550|$332.29 |$335.94 |$327.33 |
+------+----------+----------+--------+--------+--------+--------+



In [0]:
stock.printSchema()

root
 |-- Ticker: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Close/Last: string (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)



In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType,DateType
from datetime import datetime

In [0]:
# Create a user define function to conver date from string to datetype
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType,DateType
dateparser=udf(lambda date:datetime.strptime(date,"%m/%d/%Y"),DateType())
stock.withColumn("Date",dateparser("Date")).show(5)

In [0]:
def numparser(value):
  if isinstance(value,str):
    return float(value.strip("$"))
  elif isinstance(value,int) or isinstance(value,float):
    return value
  else:
      return None
    
print(numparser("100$"))

100.0


In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
parserfloat = udf(numparser,FloatType())

In [0]:
stock= stock.withColumn("Open",parserfloat("Open")).withColumn("Close",parserfloat("Close/Last")).withColumn("High",parserfloat("High")).withColumn("Low",parserfloat("Low"))

In [0]:
stock.printSchema()

root
 |-- Ticker: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Close/Last: string (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)



In [0]:
stock.show(15)

+------+----------+----------+--------+------+------+------+------+
|Ticker|      Date|Close/Last|  Volume|  Open|  High|   Low| Close|
+------+----------+----------+--------+------+------+------+------+
|  AMZN|2023-05-31|  $120.58 |72800790|121.45|122.04|119.17|120.58|
|  AMZN|2023-05-30|  $121.66 |64314810|122.37|122.92|119.86|121.66|
|  AMZN|2023-05-26|  $120.11 |96779890|116.04| 121.5|116.02|120.11|
|  AMZN|2023-05-25|  $115.00 |66496680|116.63|116.87|114.31| 115.0|
|  AMZN|2023-05-24|  $116.75 |63487940|115.35|117.34|115.02|116.75|
|  AMZN|2023-05-23|  $114.99 |67576260|114.27|117.14|113.78|114.99|
|  AMZN|2023-05-22|  $115.01 |70741120|116.77|116.77|114.25|115.01|
|  AMZN|2023-05-19|  $116.25 |55056310|118.16|118.31| 115.7|116.25|
|  AMZN|2023-05-18|  $118.15 |73174090|116.69| 118.6|116.34|118.15|
|  AMZN|2023-05-17|  $115.50 |65655180|114.89|115.83|114.22| 115.5|
|  AMZN|2023-05-16|  $113.40 |71472910|111.05|114.79|111.05| 113.4|
|  AMZN|2023-05-15|  $111.20 |53011150|111.15|11

In [0]:
from ast import Lambda
from pyspark.sql.types import IntegerType
parseint = udf (lambda volume :int(volume),IntegerType())

In [0]:
#stock=stock.withColumn("Volume",parseint("Volume"))

In [0]:
cleaned_stock=stock.select("Ticker","Date","Volume","Open","High","Low","Close")

In [0]:
cleaned_stock.printSchema()

root
 |-- Ticker: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)



In [0]:
cleaned_stock.describe().show()

+-------+------+--------------------+------------------+------------------+------------------+-----------------+
|summary|Ticker|              Volume|              Open|              High|               Low|            Close|
+-------+------+--------------------+------------------+------------------+------------------+-----------------+
|  count|   520|                 520|               520|               520|               520|              520|
|   mean|  NULL|4.7142785198076926E7|223.99860747410702|226.48809054448054|221.77086389981784|224.3515191738422|
| stddev|  NULL|4.4077822319242395E7| 99.61537484399457|100.09992568148351| 99.05107310744967|99.61753783629193|
|    min|  AAPL|             1938264|             82.24|             82.38|             81.21|            82.25|
|    max|     V|           210970800|            422.03|            422.58|           418.735|           420.18|
+-------+------+--------------------+------------------+------------------+------------------+--

In [0]:
# Calculate maximum stock Price
cleaned_stock.groupBy("Ticker").max("Open").show()

+------+---------+
|Ticker|max(Open)|
+------+---------+
| BRK-B|    331.0|
|   TSM|   103.79|
|  AAPL|   177.33|
|  META|   265.25|
|  TSLA|    200.1|
|     V|    235.0|
| GOOGL|   125.64|
|   SPY|   422.03|
|  AMZN|   122.37|
|  MSFT|   335.23|
|  NVDA|   405.95|
+------+---------+



In [0]:
cleaned_stock.groupBy("Ticker").max("Open").withColumnRenamed("max(Open)","Max_Open").show()

+------+--------+
|Ticker|Max_Open|
+------+--------+
| BRK-B|   331.0|
|   TSM|  103.79|
|  AAPL|  177.33|
|  META|  265.25|
|  TSLA|   200.1|
|     V|   235.0|
| GOOGL|  125.64|
|   SPY|  422.03|
|  AMZN|  122.37|
|  MSFT|  335.23|
|  NVDA|  405.95|
+------+--------+



In [0]:
cleaned_stock.show(5)

+------+----------+--------+------+------+------+------+
|Ticker|      Date|  Volume|  Open|  High|   Low| Close|
+------+----------+--------+------+------+------+------+
|  AMZN|2023-05-31|72800790|121.45|122.04|119.17|120.58|
|  AMZN|2023-05-30|64314810|122.37|122.92|119.86|121.66|
|  AMZN|2023-05-26|96779890|116.04| 121.5|116.02|120.11|
|  AMZN|2023-05-25|66496680|116.63|116.87|114.31| 115.0|
|  AMZN|2023-05-24|63487940|115.35|117.34|115.02|116.75|
+------+----------+--------+------+------+------+------+
only showing top 5 rows


In [0]:
import pyspark.sql.functions as func
# maximum value in each stock
cleaned_stock.groupBy("Ticker").agg(func.max("Open").alias("MAXSTOCK")).show()

+------+--------+
|Ticker|MAXSTOCK|
+------+--------+
| BRK-B|   331.0|
|   TSM|  103.79|
|  AAPL|  177.33|
|  META|  265.25|
|  TSLA|   200.1|
|     V|   235.0|
| GOOGL|  125.64|
|   SPY|  422.03|
|  AMZN|  122.37|
|  MSFT|  335.23|
|  NVDA|  405.95|
+------+--------+



In [0]:
# total volume for each stock
cleaned_stock.groupBy("Ticker").agg(func.sum("Volume").alias("Totalvolume")).show()

+------+-----------+
|Ticker|Totalvolume|
+------+-----------+
| BRK-B|  117066540|
|   TSM|  635972087|
|  AAPL| 1275364570|
|  META|  959299670|
|  TSLA| 8343485060|
|     V|  389076626|
| GOOGL| 2186269020|
|   SPY| 5050519830|
|  AMZN| 1563755070|
|  MSFT| 1079604640|
|  NVDA| 2913835190|
+------+-----------+



In [0]:
# maximum value in each stock and total volume
cleaned_stock.groupBy("Ticker").agg(func.max("Open").alias("MAXSTOCK"),func.sum("Volume").alias("Totalvolume")).show()


+------+--------+-----------+
|Ticker|MAXSTOCK|Totalvolume|
+------+--------+-----------+
| BRK-B|   331.0|  117066540|
|   TSM|  103.79|  635972087|
|  AAPL|  177.33| 1275364570|
|  META|  265.25|  959299670|
|  TSLA|   200.1| 8343485060|
|     V|   235.0|  389076626|
| GOOGL|  125.64| 2186269020|
|   SPY|  422.03| 5050519830|
|  AMZN|  122.37| 1563755070|
|  MSFT|  335.23| 1079604640|
|  NVDA|  405.95| 2913835190|
+------+--------+-----------+



In [0]:
cleaned_stock=cleaned_stock.withColumn("year",func.year("Date")).withColumn("month",func.month("Date")).withColumn("day",func.dayofmonth("Date")) 

In [0]:
cleaned_stock.show(5)

+------+----------+--------+------+------+------+------+----+-----+---+
|Ticker|      Date|  Volume|  Open|  High|   Low| Close|year|month|day|
+------+----------+--------+------+------+------+------+----+-----+---+
|  AMZN|2023-05-31|72800790|121.45|122.04|119.17|120.58|2023|    5| 31|
|  AMZN|2023-05-30|64314810|122.37|122.92|119.86|121.66|2023|    5| 30|
|  AMZN|2023-05-26|96779890|116.04| 121.5|116.02|120.11|2023|    5| 26|
|  AMZN|2023-05-25|66496680|116.63|116.87|114.31| 115.0|2023|    5| 25|
|  AMZN|2023-05-24|63487940|115.35|117.34|115.02|116.75|2023|    5| 24|
+------+----------+--------+------+------+------+------+----+-----+---+
only showing top 5 rows


In [0]:
#calcualte the maximum and minimum value for each Ticker per YEAR
cleaned_stock.groupBy("Ticker","year").agg(func.max("Open").alias("yearlyhigh"),func.min("open").alias("lowyear")).show()

+------+----+----------+-------+
|Ticker|year|yearlyhigh|lowyear|
+------+----+----------+-------+
| BRK-B|2023|     331.0| 307.85|
|     V|2023|     235.0| 214.41|
|   SPY|2023|    422.03| 381.81|
| GOOGL|2023|    125.64|  90.09|
|  NVDA|2023|    405.95| 227.52|
|  META|2023|    265.25| 207.24|
|  MSFT|2023|    335.23| 279.51|
|   TSM|2023|    103.79|  82.24|
|  AMZN|2023|    122.37| 101.47|
|  TSLA|2023|     200.1| 152.64|
|  AAPL|2023|    177.33| 164.89|
+------+----+----------+-------+



In [0]:
#calcualte the maximum and minimum value for each Ticker per YEAR
cleaned_stock.groupBy("Ticker","year","month").agg(func.max("Open").alias("month"),func.min("open").alias("lowmonth")).show(5)

+------+----+-----+------+--------+
|Ticker|year|month| month|lowmonth|
+------+----+-----+------+--------+
|  TSLA|2023|    4|199.91|  152.64|
|  META|2023|    4|239.89|  208.84|
|  NVDA|2023|    5|405.95|  276.51|
|  NVDA|2023|    3|272.29|  227.52|
|   TSM|2023|    5|103.79|   82.62|
+------+----+-----+------+--------+
only showing top 5 rows


In [0]:
monthlyanalysis=cleaned_stock.groupBy("Ticker","year","month").agg(func.max("Open").alias("highmonth"),func.min("open").alias("lowmonth"))
monthlyanalysis.show(5)

+------+----+-----+---------+--------+
|Ticker|year|month|highmonth|lowmonth|
+------+----+-----+---------+--------+
|  TSLA|2023|    4|   199.91|  152.64|
|  META|2023|    4|   239.89|  208.84|
|  NVDA|2023|    5|   405.95|  276.51|
|  NVDA|2023|    3|   272.29|  227.52|
|   TSM|2023|    5|   103.79|   82.62|
+------+----+-----+---------+--------+
only showing top 5 rows


In [0]:
historicalAnalysis=cleaned_stock.join(monthlyanalysis,(cleaned_stock.year==monthlyanalysis.year) &(cleaned_stock.month==monthlyanalysis.month)& (cleaned_stock.Ticker==monthlyanalysis.Ticker),how="inner").drop(monthlyanalysis.year,monthlyanalysis.month,monthlyanalysis.Ticker )

In [0]:
historicalAnalysis.show(5)

+----------+--------+------+------+------+------+---+------+----+-----+---------+--------+
|      Date|  Volume|  Open|  High|   Low| Close|day|Ticker|year|month|highmonth|lowmonth|
+----------+--------+------+------+------+------+---+------+----+-----+---------+--------+
|2023-05-31|72800790|121.45|122.04|119.17|120.58| 31|  AMZN|2023|    5|   122.37|  101.47|
|2023-05-30|64314810|122.37|122.92|119.86|121.66| 30|  AMZN|2023|    5|   122.37|  101.47|
|2023-05-26|96779890|116.04| 121.5|116.02|120.11| 26|  AMZN|2023|    5|   122.37|  101.47|
|2023-05-25|66496680|116.63|116.87|114.31| 115.0| 25|  AMZN|2023|    5|   122.37|  101.47|
|2023-05-24|63487940|115.35|117.34|115.02|116.75| 24|  AMZN|2023|    5|   122.37|  101.47|
+----------+--------+------+------+------+------+---+------+----+-----+---------+--------+
only showing top 5 rows


SQL Queries using spark

In [0]:
finalstocks=historicalAnalysis.select(["Ticker","high","low","year","month","open","close","highmonth","lowmonth","Date"]).createOrReplaceTempView("stockData")

In [0]:
spark.sql("select * from stockData where Ticker='AAPL'").show(15)

+------+------+------+----+-----+------+------+---------+--------+----------+
|Ticker|  high|   low|year|month|  open| close|highmonth|lowmonth|      Date|
+------+------+------+----+-----+------+------+---------+--------+----------+
|  AAPL|179.35|176.76|2023|    5|177.33|177.25|   177.33|  164.89|2023-05-31|
|  AAPL|178.99|176.57|2023|    5|176.96| 177.3|   177.33|  164.89|2023-05-30|
|  AAPL|175.77|173.11|2023|    5|173.32|175.43|   177.33|  164.89|2023-05-26|
|  AAPL| 173.9|171.69|2023|    5|172.41|172.99|   177.33|  164.89|2023-05-25|
|  AAPL|172.42|170.52|2023|    5|171.09|171.84|   177.33|  164.89|2023-05-24|
|  AAPL|173.38|171.28|2023|    5|173.13|171.56|   177.33|  164.89|2023-05-23|
|  AAPL|174.71|173.45|2023|    5|173.98| 174.2|   177.33|  164.89|2023-05-22|
|  AAPL|176.39|174.94|2023|    5|176.39|175.16|   177.33|  164.89|2023-05-19|
|  AAPL|175.24|172.58|2023|    5| 173.0|175.05|   177.33|  164.89|2023-05-18|
|  AAPL|172.93|170.42|2023|    5|171.71|172.69|   177.33|  164.8

In [0]:
cleaned_stock.show(5)

+------+----------+--------+------+------+------+------+----+-----+---+
|Ticker|      Date|  Volume|  Open|  High|   Low| Close|year|month|day|
+------+----------+--------+------+------+------+------+----+-----+---+
|  AMZN|2023-05-31|72800790|121.45|122.04|119.17|120.58|2023|    5| 31|
|  AMZN|2023-05-30|64314810|122.37|122.92|119.86|121.66|2023|    5| 30|
|  AMZN|2023-05-26|96779890|116.04| 121.5|116.02|120.11|2023|    5| 26|
|  AMZN|2023-05-25|66496680|116.63|116.87|114.31| 115.0|2023|    5| 25|
|  AMZN|2023-05-24|63487940|115.35|117.34|115.02|116.75|2023|    5| 24|
+------+----------+--------+------+------+------+------+----+-----+---+
only showing top 5 rows


In [0]:
snapshot=cleaned_stock.select(["Ticker","Date","Open"])
snapshot.show(5)

+------+----------+------+
|Ticker|      Date|  Open|
+------+----------+------+
|  AMZN|2023-05-31|121.45|
|  AMZN|2023-05-30|122.37|
|  AMZN|2023-05-26|116.04|
|  AMZN|2023-05-25|116.63|
|  AMZN|2023-05-24|115.35|
+------+----------+------+
only showing top 5 rows


In [0]:
from pyspark.sql.window import Window
lag1day=Window.partitionBy("Ticker").orderBy("Date")
snapshot.withColumn("previousopen",func.lag("Open",1).over(lag1day)).show(5)

+------+----------+------+------------+
|Ticker|      Date|  Open|previousopen|
+------+----------+------+------------+
|  AAPL|2023-05-01|169.28|        NULL|
|  AAPL|2023-05-02|170.09|      169.28|
|  AAPL|2023-05-03| 169.5|      170.09|
|  AAPL|2023-05-04|164.89|       169.5|
|  AAPL|2023-05-05|170.98|      164.89|
+------+----------+------+------------+
only showing top 5 rows


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, max

maximumstock = Window.partitionBy("Ticker").orderBy(snapshot.Open.desc())

In [0]:
snapshot.withColumn("maxopen",func.row_number().over(maximumstock)).show(10)

+------+----------+------+-------+
|Ticker|      Date|  Open|maxopen|
+------+----------+------+-------+
|  AAPL|2023-05-31|177.33|      1|
|  AAPL|2023-05-30|176.96|      2|
|  AAPL|2023-05-19|176.39|      3|
|  AAPL|2023-05-22|173.98|      4|
|  AAPL|2023-05-11|173.85|      5|
|  AAPL|2023-05-12|173.62|      6|
|  AAPL|2023-05-26|173.32|      7|
|  AAPL|2023-05-15|173.16|      8|
|  AAPL|2023-05-23|173.13|      9|
|  AAPL|2023-05-09|173.05|     10|
+------+----------+------+-------+
only showing top 10 rows


In [0]:
Result=snapshot.withColumn("maxopen",func.row_number().over(maximumstock)).filter(col("maxopen")<5).show(10)

+------+----------+------+-------+
|Ticker|      Date|  Open|maxopen|
+------+----------+------+-------+
|  AAPL|2023-05-31|177.33|      1|
|  AAPL|2023-05-30|176.96|      2|
|  AAPL|2023-05-19|176.39|      3|
|  AAPL|2023-05-22|173.98|      4|
|  AMZN|2023-05-30|122.37|      1|
|  AMZN|2023-05-31|121.45|      2|
|  AMZN|2023-05-19|118.16|      3|
|  AMZN|2023-05-22|116.77|      4|
| BRK-B|2023-05-19| 331.0|      1|
| BRK-B|2023-05-22|330.75|      2|
+------+----------+------+-------+
only showing top 10 rows


In [0]:
cleaned_stock.write.mode("overwrite").option("header", "true").csv("/mnt/data/stock_prices")